# Basalt Prompt SDK Demo

This notebook demonstrates how to use the Basalt Prompt SDK to interact with your Basalt prompts.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Needed to make notebook work in VSCode

os.environ["BASALT_BUILD"] = "development"

from basalt import Basalt

# Initialize the SDK
basalt = Basalt(
	api_key="sk-d4ef...", # Replace with your API key
	log_level="debug" # Optional: Set log level
)

## 1. Getting a Prompt

Retrieve a specific prompt using a slug, with optional filters for tag and version.

In [ ]:
# Get a prompt by slug (default is production version)
error, result = basalt.prompt.get('prompt-slug')

if error:
    print(f"Error fetching prompt: {error}")
else:
    print(f"Successfully fetched prompt: {result.prompt[:100]}...")

## 2. Using Tags and Versions

You can specify a tag or version when retrieving a prompt.

In [ ]:
# Get a prompt with a specific tag
error, result_tag = basalt.prompt.get(slug='prompt-slug', tag='latest')

if error:
    print(f"Error fetching prompt with tag: {error}")
else:
    print(f"Successfully fetched prompt with tag 'latest': {result_tag.prompt[:100]}...")

# Get a prompt with a specific version
error, result_version = basalt.prompt.get(slug='prompt-slug', version='1.0.0')

if error:
    print(f"Error fetching prompt with version: {error}")
else:
    print(f"Successfully fetched prompt with version '1.0.0': {result_version.prompt[:100]}...")

## 3. Using Variables

If your prompt has variables, you can pass them when fetching the prompt.

In [ ]:
# Get a prompt with variables
error, result_vars = basalt.prompt.get(
    slug='prompt-slug-with-vars', 
    variables={
        'name': 'John Doe',
        'role': 'Developer',
        'company': 'Acme Inc'
    }
)

if error:
    print(f"Error fetching prompt with variables: {error}")
else:
    print(f"Successfully fetched prompt with variables: {result_vars.prompt[:100]}...")

## 4. Using Prompts with AI Providers

Once you have retrieved a prompt, you can use it with your AI provider of choice.

In [ ]:
# Example with OpenAI (you'll need to install the openai package)
try:
    import openai
    
    # Set up OpenAI client
    client = openai.OpenAI(api_key="your-openai-api-key")  # Replace with your OpenAI API key
    
    # Get a prompt from Basalt
    error, result = basalt.prompt.get('prompt-slug')
    
    if error:
        print(f"Error fetching prompt: {error}")
    else:
        # Use the prompt with OpenAI
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": result.prompt}
            ]
        )
        
        print(f"OpenAI Response: {response.choices[0].message.content}")
except ImportError:
    print("OpenAI package not installed. Install with: pip install openai")

## 5. Error Handling

Proper error handling when working with prompts.

In [ ]:
# Example of handling different error scenarios
def get_prompt_safely(slug, tag=None, version=None, variables=None):
    try:
        error, result = basalt.prompt.get(
            slug=slug,
            tag=tag,
            version=version,
            variables=variables
        )
        
        if error:
            print(f"Error fetching prompt '{slug}': {error}")
            return None
        return result.prompt
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return None

# Test with a non-existent prompt
prompt_text = get_prompt_safely("non-existent-prompt")
print(f"Result: {prompt_text}")

# Test with a valid prompt
prompt_text = get_prompt_safely("prompt-slug")
if prompt_text:
    print(f"Valid prompt retrieved: {prompt_text[:50]}...")

## 6. Integration with Monitor SDK

You can use the Prompt SDK together with the Monitor SDK for a complete workflow.

In [ ]:
# Create a trace
trace = basalt.monitor.create_trace(
    "prompt-workflow",
    {
        "input": "Tell me about artificial intelligence",
        "user": {"id": "user123", "name": "Jane Smith"},
        "metadata": {"source": "web"}
    }
)

# Create a log for processing
processing_log = trace.create_log({
    "type": "span",
    "name": "ai-response-generation",
    "input": trace.input
})

# Get a prompt from Basalt
error, prompt_result = basalt.prompt.get('ai-explanation-prompt')

if error:
    processing_log.end({"error": str(error)})
    trace.end({"status": "error", "message": f"Failed to get prompt: {error}"})
else:
    # Create a generation using the retrieved prompt
    generation = processing_log.create_generation({
        "name": "ai-explanation",
        "input": trace.input,
        "prompt": prompt_result.prompt
    })
    
    # Simulate AI response
    ai_response = """
    Artificial Intelligence (AI) refers to systems or machines that mimic human intelligence
    to perform tasks and can iteratively improve themselves based on the information they collect.
    AI manifests in a number of forms including:
    
    1. Machine Learning
    2. Natural Language Processing
    3. Computer Vision
    4. Robotics
    """
    
    # End the generation
    generation.end(ai_response)
    
    # End the log and trace
    processing_log.end({"status": "success", "output": ai_response})
    trace.end({"status": "success"})
    
    print(f"Generated response using Basalt prompt:\n{ai_response}")